# Header

In [2]:
import pandas as pd
import time
import numpy as np

# Metrics
from sklearn.metrics import accuracy_score, f1_score

In [3]:
from utils import train_val_split
from utils import train_datapath, test_datapath

targets_for_test_df = pd.read_csv('data/targets_for_test.csv', index_col=0)

In [4]:
# Define function to create sequences
def create_sequences(data, num_timesteps):
    sequences = []
    for i in range(len(data) - num_timesteps + 1):
        sequences.append(data[i:i+num_timesteps])
    return np.array(sequences)


# Base RNN

## Crude

In [6]:
train_df = pd.read_csv(train_datapath)
test_df = pd.read_csv(test_datapath)

In [7]:
from keras.models import Sequential
from keras.layers import SimpleRNN, Dense
from sklearn.preprocessing import StandardScaler
import tensorflow_addons as tfa


start_time = time.time()


# Extract features and target
X_train, y_train, X_val, y_val = train_val_split(train_df)

# Scale the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)



# Reshape data to fit RNN input requirements (samples, time steps, features)
num_timesteps = 15
num_features = X_train.shape[1]  # Assuming X_train has 10 features

# Generate sequences
X_train_seq = create_sequences(X_train, num_timesteps)
X_val_seq = create_sequences(X_val, num_timesteps)

# Adjust y_train and y_val accordingly
y_train_seq = y_train[num_timesteps - 1:]
y_val_seq = y_val[num_timesteps - 1:]

# Define the RNN model
model = Sequential()
model.add(SimpleRNN(50, input_shape=(X_train_seq.shape[1], X_train_seq.shape[2])))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[tfa.metrics.F1Score(num_classes=1, threshold=0.5)])

# Train the model
model.fit(X_train_seq, y_train_seq, epochs=10, batch_size=32)

# Predict probabilities
y_pred_prob = model.predict(X_val_seq)

# Convert probabilities to binary predictions
y_pred = (y_pred_prob > 0.5).astype(int)


end_time = time.time()
minutes = (end_time - start_time) // 60
seconds = (end_time - start_time) % 60
print(f'Time elapsed: {minutes:.0f}m {seconds:.1f}s')
print('--------------------------------------')
# Calculate accuracy
accuracy = accuracy_score(y_val_seq, y_pred)
print(f'Validation Accuracy: {accuracy:.5f}')

# Calculate F1 macro score
f1_macro = f1_score(y_val_seq, y_pred, average='macro')
print(f'Validation F1 Macro Score: {f1_macro:.5f}')


d:\01_GitHub\Directional-Forecasting-in-Cryptocurrencies\env_rnn\Lib\site-packages\tensorflow_addons\utils\tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


Epoch 1/10
53061/53061 [==============================] - 172s 3ms/step - loss: 0.6924 - f1_score: 0.2087
Epoch 2/10
53061/53061 [==============================] - 171s 3ms/step - loss: 0.6924 - f1_score: 0.2636
Epoch 3/10
53061/53061 [==============================] - 184s 3ms/step - loss: 0.6927 - f1_score: 0.2897
Epoch 4/10
53061/53061 [==============================] - 184s 3ms/step - loss: 0.6927 - f1_score: 0.2892
Epoch 5/10
53061/53061 [==============================] - 194s 4ms/step - loss: 0.6926 - f1_score: 0.2970
Epoch 6/10
53061/53061 [==============================] - 210s 4ms/step - loss: 0.6927 - f1_score: 0.2947
Epoch 7/10
53061/53061 [==============================] - 221s 4ms/step - loss: 0.6927 - f1_score: 0.2983
Epoch 8/10
53061/53061 [==============================] - 204s 4ms/step - loss: 0.6927 - f1_score: 0.2957
Epoch 9/10
53061/53061 [==============================] - 186s 4ms/step - loss: 0.6928 - f1_score: 0.2841
Epoch 10/10
13265/13265 [=====================

In [10]:
# Ensure the test data is preprocessed in the same way as the training data
X_test = test_df.drop(columns=['row_id'])
X_test = scaler.transform(X_test)

# Reshape test data to fit RNN input requirements (samples, time steps, features)
X_test_seq = create_sequences(X_test, num_timesteps)

# Make predictions on the test data
test_predictions_prob = model.predict(X_test_seq)

# Convert probabilities to binary predictions
test_predictions = (test_predictions_prob > 0.5).astype('float32')

# Fil the predictions to match the length of the targets_for_test_df
filled_test_predictions = (len(targets_for_test_df) - len(test_predictions)) * [0]  + test_predictions.flatten().tolist()

# Calculate accuracy
accuracy = accuracy_score(targets_for_test_df, filled_test_predictions)
print(f'Test Accuracy: {accuracy:.5f}')

# Calculate F1 macro score
f1_macro = f1_score(targets_for_test_df, filled_test_predictions, average='macro')
print(f'Test F1 Macro Score: {f1_macro:.5f}')

28426/28426 [==============================] - 59s 2ms/step
Test Accuracy: 0.57593
Test F1 Macro Score: 0.43709


In [ ]:
# Create a new DataFrame for the submission
submission_df = pd.DataFrame({
    'row_id': test_df['row_id'],
    'target': filled_test_predictions
})

# Save the submission file
submission_df.to_csv('submission.csv', index=False)
print("Predictions saved to submission.csv")



ValueError: in user code:

    File "d:\01_GitHub\Directional-Forecasting-in-Cryptocurrencies\env_rnn\Lib\site-packages\keras\engine\training.py", line 2169, in predict_function  *
        return step_function(self, iterator)
    File "d:\01_GitHub\Directional-Forecasting-in-Cryptocurrencies\env_rnn\Lib\site-packages\keras\engine\training.py", line 2155, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "d:\01_GitHub\Directional-Forecasting-in-Cryptocurrencies\env_rnn\Lib\site-packages\keras\engine\training.py", line 2143, in run_step  **
        outputs = model.predict_step(data)
    File "d:\01_GitHub\Directional-Forecasting-in-Cryptocurrencies\env_rnn\Lib\site-packages\keras\engine\training.py", line 2111, in predict_step
        return self(x, training=False)
    File "d:\01_GitHub\Directional-Forecasting-in-Cryptocurrencies\env_rnn\Lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_t

In [ ]:
del train_df, test_df, X_train, y_train, X_val, y_val, X_train_seq, X_val_seq, y_train_seq, y_val_seq, model, y_pred_prob, y_pred, X_test, X_test_seq, test_predictions_prob, test_predictions

## SVD Features

In [ ]:
svd_train = pd.read_csv('data/svd_train.csv')
svd_test = pd.read_csv('data/svd_test.csv')

In [ ]:
from keras.models import Sequential
from keras.layers import SimpleRNN, Dense
from sklearn.preprocessing import StandardScaler
import tensorflow_addons as tfa


start_time = time.time()


# Extract features and target
X_train, y_train, X_val, y_val = train_val_split(svd_train)

# Scale the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)



# Reshape data to fit RNN input requirements (samples, time steps, features)
num_timesteps = 15
num_features = X_train.shape[1]  # Assuming X_train has 10 features

# Generate sequences
X_train_seq = create_sequences(X_train, num_timesteps)
X_val_seq = create_sequences(X_val, num_timesteps)

# Adjust y_train and y_val accordingly
y_train_seq = y_train[num_timesteps - 1:]
y_val_seq = y_val[num_timesteps - 1:]

# Define the RNN model
model = Sequential()
model.add(SimpleRNN(50, input_shape=(X_train_seq.shape[1], X_train_seq.shape[2])))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[tfa.metrics.F1Score(num_classes=1, threshold=0.5)])

# Train the model
model.fit(X_train_seq, y_train_seq, epochs=10, batch_size=32)

# Predict probabilities
y_pred_prob = model.predict(X_val_seq)

# Convert probabilities to binary predictions
y_pred = (y_pred_prob > 0.5).astype(int)


end_time = time.time()
minutes = (end_time - start_time) // 60
seconds = (end_time - start_time) % 60
print(f'Time elapsed: {minutes:.0f}m {seconds:.1f}s')
print('--------------------------------------')
# Calculate accuracy
accuracy = accuracy_score(y_val_seq, y_pred)
print(f'Validation Accuracy: {accuracy:.5f}')

# Calculate F1 macro score
f1_macro = f1_score(y_val_seq, y_pred, average='macro')
print(f'Validation F1 Macro Score: {f1_macro:.5f}')


Epoch 1/10
52919/52919 [==============================] - 178s 3ms/step - loss: 0.6905 - f1_score: 0.3922
Epoch 2/10
52919/52919 [==============================] - 178s 3ms/step - loss: 0.6887 - f1_score: 0.4011
Epoch 3/10
52919/52919 [==============================] - 174s 3ms/step - loss: 0.6881 - f1_score: 0.4064
Epoch 4/10
52919/52919 [==============================] - 174s 3ms/step - loss: 0.6874 - f1_score: 0.4103
Epoch 5/10
52919/52919 [==============================] - 175s 3ms/step - loss: 0.6874 - f1_score: 0.4114
Epoch 6/10
52919/52919 [==============================] - 175s 3ms/step - loss: 0.6874 - f1_score: 0.4158
Epoch 7/10
52919/52919 [==============================] - 176s 3ms/step - loss: 0.6873 - f1_score: 0.4157
Epoch 8/10
52919/52919 [==============================] - 177s 3ms/step - loss: 0.6871 - f1_score: 0.4153
Epoch 9/10
52919/52919 [==============================] - 175s 3ms/step - loss: 0.6871 - f1_score: 0.4152
Epoch 10/10
13230/13230 [=====================

In [ ]:
# Ensure the test data is preprocessed in the same way as the training data
X_SVD_test = svd_test.drop(columns=['row_id'])
X_SVD_test = scaler.transform(X_SVD_test)

# Reshape test data to fit RNN input requirements (samples, time steps, features)
X_SVD_test_seq = create_sequences(X_SVD_test, num_timesteps)

# Make predictions on the test data
test_predictions_prob = model.predict(X_SVD_test_seq)

# Convert probabilities to binary predictions
test_predictions = (test_predictions_prob > 0.5).astype(int)

filled_test_predictions = (len(targets_for_test_df) - len(test_predictions)) * [0]  + test_predictions.flatten().tolist()

# Calculate accuracy
accuracy = accuracy_score(targets_for_test_df, filled_test_predictions)
print(f'Test Accuracy: {accuracy:.5f}')

# Calculate F1 macro score
f1_macro = f1_score(targets_for_test_df, filled_test_predictions, average='macro')
print(f'Test F1 Macro Score: {f1_macro:.5f}')

28423/28423 [==============================] - 54s 2ms/step
Test Accuracy: 0.46368
Test F1 Macro Score: 0.45448


In [ ]:

# Create a new DataFrame for the submission
submission_df = pd.DataFrame({
    'row_id': test_df['row_id'],
    'target': filled_test_predictions
})

# Save the submission file
submission_df.to_csv('submission.csv', index=False)
print("Predictions saved to submission.csv")



In [ ]:
del svd_train, svd_test, X_train, y_train, X_val, y_val, X_train_seq, X_val_seq, y_train_seq, y_val_seq, model, y_pred_prob, y_pred, X_test, X_test_seq, test_predictions_prob, test_predictions

## New Features

In [ ]:
treated_train_df = pd.read_csv('data/treated_train.csv')
treated_test_df = pd.read_csv('data/treated_test.csv')

In [ ]:
from keras.models import Sequential
from keras.layers import SimpleRNN, Dense
from sklearn.preprocessing import StandardScaler
import tensorflow_addons as tfa


start_time = time.time()


# Extract features and target
X_train, y_train, X_val, y_val = train_val_split(treated_train_df)

# Scale the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)



# Reshape data to fit RNN input requirements (samples, time steps, features)
num_timesteps = 15
num_features = X_train.shape[1]  # Assuming X_train has 10 features

# Generate sequences
X_train_seq = create_sequences(X_train, num_timesteps)
X_val_seq = create_sequences(X_val, num_timesteps)

# Adjust y_train and y_val accordingly
y_train_seq = y_train[num_timesteps - 1:]
y_val_seq = y_val[num_timesteps - 1:]

# Define the RNN treated_model
treated_model = Sequential()
treated_model.add(SimpleRNN(50, input_shape=(X_train_seq.shape[1], X_train_seq.shape[2])))
treated_model.add(Dense(1, activation='sigmoid'))
treated_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[tfa.metrics.F1Score(num_classes=1, threshold=0.5)])

# Train the treated_model
treated_model.fit(X_train_seq, y_train_seq, epochs=10, batch_size=32)

# Predict probabilities
y_pred_prob = treated_model.predict(X_val_seq)

# Convert probabilities to binary predictions
y_pred = (y_pred_prob > 0.5).astype(int)


end_time = time.time()
minutes = (end_time - start_time) // 60
seconds = (end_time - start_time) % 60
print(f'Time elapsed: {minutes:.0f}m {seconds:.1f}s')
print('--------------------------------------')
# Calculate accuracy
accuracy = accuracy_score(y_val_seq, y_pred)
print(f'Validation Accuracy: {accuracy:.5f}')

# Calculate F1 macro score
f1_macro = f1_score(y_val_seq, y_pred, average='macro')
print(f'Validation F1 Macro Score: {f1_macro:.5f}')


MemoryError: Unable to allocate 284. MiB for an array with shape (1693399, 22) and data type float64

In [ ]:
# Ensure the test data is preprocessed in the same way as the training data
X_test = treated_test_df.drop(columns=['row_id'])
X_test = scaler.transform(X_test)

# Reshape test data to fit RNN input requirements (samples, time steps, features)
X_test_seq = create_sequences(X_test, num_timesteps)

# Make predictions on the test data
test_predictions_prob = model.predict(X_test_seq)

# Convert probabilities to binary predictions
test_predictions = (test_predictions_prob > 0.5).astype(int)

filled_test_predictions = (len(targets_for_test_df) - len(test_predictions)) * [0]  + test_predictions.flatten().tolist()

# Calculate accuracy
accuracy = accuracy_score(targets_for_test_df, filled_test_predictions)
print(f'Test Accuracy: {accuracy:.5f}')

# Calculate F1 macro score
f1_macro = f1_score(targets_for_test_df, filled_test_predictions, average='macro')
print(f'Test F1 Macro Score: {f1_macro:.5f}')

In [ ]:
del treated_train_df, treated_test_df, X_train, y_train, X_val, y_val, X_train_seq, X_val_seq, y_train_seq, y_val_seq, treated_model, y_pred_prob, y_pred, X_test, X_test_seq, test_predictions_prob, test_predictions

## Only new features

In [ ]:
new_features_train_df = pd.read_csv('data/new_features_train.csv')
new_features_test_df = pd.read_csv('data/new_features_test.csv')

In [ ]:
from keras.models import Sequential
from keras.layers import SimpleRNN, Dense
from sklearn.preprocessing import StandardScaler
import tensorflow_addons as tfa


start_time = time.time()


# Extract features and target
X_train, y_train, X_val, y_val = train_val_split(new_features_train_df)

# Scale the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)



# Reshape data to fit RNN input requirements (samples, time steps, features)
num_timesteps = 15
num_features = X_train.shape[1]  # Assuming X_train has 10 features

# Generate sequences
X_train_seq = create_sequences(X_train, num_timesteps)
X_val_seq = create_sequences(X_val, num_timesteps)

# Adjust y_train and y_val accordingly
y_train_seq = y_train[num_timesteps - 1:]
y_val_seq = y_val[num_timesteps - 1:]

# Define the RNN only_model
only_model = Sequential()
only_model.add(SimpleRNN(50, input_shape=(X_train_seq.shape[1], X_train_seq.shape[2])))
only_model.add(Dense(1, activation='sigmoid'))
only_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[tfa.metrics.F1Score(num_classes=1, threshold=0.5)])

# Train the only_model
only_model.fit(X_train_seq, y_train_seq, epochs=10, batch_size=32)

# Predict probabilities
y_pred_prob = only_model.predict(X_val_seq)

# Convert probabilities to binary predictions
y_pred = (y_pred_prob > 0.5).astype(int)


end_time = time.time()
minutes = (end_time - start_time) // 60
seconds = (end_time - start_time) % 60
print(f'Time elapsed: {minutes:.0f}m {seconds:.1f}s')
print('--------------------------------------')
# Calculate accuracy
accuracy = accuracy_score(y_val_seq, y_pred)
print(f'Validation Accuracy: {accuracy:.5f}')

# Calculate F1 macro score
f1_macro = f1_score(y_val_seq, y_pred, average='macro')
print(f'Validation F1 Macro Score: {f1_macro:.5f}')


array([[1],
       [0],
       [0],
       ...,
       [0],
       [0],
       [0]])

In [ ]:
# Ensure the test data is preprocessed in the same way as the training data
X_test = new_features_test_df.drop(columns=['row_id'])
X_test = scaler.transform(X_test)

# Reshape test data to fit RNN input requirements (samples, time steps, features)
X_test_seq = create_sequences(X_test, num_timesteps)

# Make predictions on the test data
test_predictions_prob = model.predict(X_test_seq)

# Convert probabilities to binary predictions
test_predictions = (test_predictions_prob > 0.5).astype(int)

filled_test_predictions = (len(targets_for_test_df) - len(test_predictions)) * [0]  + test_predictions.flatten().tolist()

# Calculate accuracy
accuracy = accuracy_score(targets_for_test_df, filled_test_predictions)
print(f'Test Accuracy: {accuracy:.5f}')

# Calculate F1 macro score
f1_macro = f1_score(targets_for_test_df, filled_test_predictions, average='macro')
print(f'Test F1 Macro Score: {f1_macro:.5f}')